In [7]:
using JuMP
using CPLEX
Pkg.add("OffsetArrays") # -> Nos permitirá usar matrices que empiecen con índice 0 
using OffsetArrays

INFO: Cloning cache of OffsetArrays from https://github.com/JuliaArrays/OffsetArrays.jl.git
INFO: Installing OffsetArrays v0.3.0
INFO: Package database updated
INFO: METADATA is out-of-date — you may not have the latest version of OffsetArrays
INFO: Use `Pkg.update()` to get the latest versions of your packages
INFO: Precompiling module OffsetArrays.


# Programación dinámica

Veremos únicamente casos que se calificarían como programación dinámica determinista en horizonte finito (categoría en la que caen la mayoría de problemas de optimización combinatoria determinista).

Utilizaremos dos ejemplos (knapsack y lot sizing) que ya hemos visto y que luego intentaremos combinar/comparar con los métodos expuestos con anterioridad. 

No voy a repasar los fundamentos de la programación dinámica. Si alguien necesita un repaso más allá del nivel introductorio, recomiendo el libro de Denardo porque es uno de los más accesibles y baratos (E. Denardo, Dynamic Programming: Models and Applications).

El problema de knapsack es posiblemente uno de los modelos clásicos más simples que hay. Ofrece una formulación de programación entera simple que además ha sido muy tratada en la literatura. Antes de nada veamos un modelo (sacado de la documentación de JuMP https://github.com/JuliaOpt/JuMP.jl/blob/master/examples/knapsack.jl)

Trataremos la versión 0-1 que es la que aparece en el modelo y luego veremos los cambios necesarios para tratar la versión general (que usamos en el ejemplo de column generation)

In [2]:
m = Model(solver=CplexSolver())

@variable(m, x[1:5], Bin)

profit = [ 5, 3, 2, 7, 4 ]
weight = [ 2, 8, 4, 2, 5 ]
capacity = 10

# Objective: maximize profit
@objective(m, Max, dot(profit, x))

# Constraint: can carry all
@constraint(m, dot(weight, x) <= capacity)

# Solve problem using MIP solver
status = solve(m)

println("Objective is: ", getobjectivevalue(m))
println("Solution is:")
for i = 1:5
    print("x[$i] = ", getvalue(x[i]),"\n")
end

Found incumbent of value 0.000000 after 0.00 sec. (0.00 ticks)
Tried aggregator 1 time.
MIP Presolve added 1 rows and 1 columns.
MIP Presolve modified 5 coefficients.
Reduced MIP has 2 rows, 6 columns, and 10 nonzeros.
Reduced MIP has 5 binaries, 1 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.01 ticks)
Probing fixed 0 vars, tightened 1 bounds.
Probing time = 0.00 sec. (0.00 ticks)
Tried aggregator 1 time.
MIP Presolve eliminated 1 rows and 1 columns.
MIP Presolve added 1 rows and 1 columns.
Reduced MIP has 2 rows, 6 columns, and 10 nonzeros.
Reduced MIP has 5 binaries, 1 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.01 ticks)
Probing time = 0.00 sec. (0.00 ticks)
Clique table members: 2.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 2 threads.
Root relaxation solution time = 0.00 sec. (0.00 ticks)

        Nodes                                         Cuts/
   Node  L

## 0-1 Knapsack. Método 1. Tabla completa

Cada etapa (columna de la tabla) corresponde a un item. Los estados representan la disponibilidad.

No se muestra cómo reconstruir los valores porque es algo tedioso y la verdad no muy útil como se verá en la siguiente implementación

In [8]:
function knapsack_m1(p,w,c)
    T=size(p,1)
    tableProfit = OffsetArray(Int64, 1:T, 0:c)
    for t in 1:T
        for s in 0:c
            tableProfit[t,s]=0
        end
    end
    #etapa 1
    tableProfit[1,0]=0
    tableProfit[1,w[1]]=p[1]
    #etapa 2 to T
    for t in 2:T
        s=c-w[t]
        while s>=0
            if tableProfit[t-1,s]>0 
                if tableProfit[t,s] < tableProfit[t-1,s]
                    tableProfit[t,s] = tableProfit[t-1,s]
                end
                if tableProfit[t,s+w[t]]<(tableProfit[t-1,s]+p[t])
                    tableProfit[t,s+w[t]]=tableProfit[t-1,s]+p[t]
                end
            end
            s -= 1
        end
    end
    beneficio=0
    maxLoad=0
    previous=0
    for s in 0:c
        if beneficio < tableProfit[T,s]
            beneficio=tableProfit[T,s]
        end
    end
    return beneficio
end

knapsack_m1 (generic function with 1 method)

In [9]:
beneficio = knapsack_m1(profit,weight,capacity)
println(beneficio)

16


## 0-1 Knapsack. Método 2. Tabla reducida

En realidad no es necesario tener en cuenta cada etapa por separado. Veremos que es suficiente con vectores si vamos con cuidado en el orden de las operaciones

In [22]:
function knapsack_m2(p,w,c)
    T=size(p,1)
    Profit = OffsetArray(Int64, 0:c)
    Path = OffsetArray(Int64, 0:c)
    Profit[0]=0
    Path[0]=0
    for i in 1:c
        Profit[i]= -1
    end
    for i in 1:c
        Path[i]= -1
    end
    for t in 1:T
        s=c-w[t]
        while s>=0
            #first check if current assignment is feasible
            if Path[s]>=0 #first check if current assignment is feasible
                if Profit[s+w[t]]< (Profit[s]+p[t])
                    Profit[s+w[t]]=Profit[s]+p[t]
                    Path[s+w[t]]=t
                end
            end
            s -= 1
        end
        println(t,"\t",Profit,"\t\t",Path)
    end
    ySol=zeros(Int64,T)
    #first find position, then backtrack
    return maximum(Profit)
end

#profit = [ 5, 3, 2, 7, 4 ]
#weight = [ 2, 8, 4, 2, 5 ]


knapsack_m2 (generic function with 1 method)

In [23]:
beneficio, ySol = knapsack_m2(profit,weight,capacity)
println(beneficio)

1	[0,-1,5,-1,-1,-1,-1,-1,-1,-1,-1]		[0,-1,1,-1,-1,-1,-1,-1,-1,-1,-1]
2	[0,-1,5,-1,-1,-1,-1,-1,3,-1,8]		[0,-1,1,-1,-1,-1,-1,-1,2,-1,2]
3	[0,-1,5,-1,2,-1,7,-1,3,-1,8]		[0,-1,1,-1,3,-1,3,-1,2,-1,2]
4	[0,-1,7,-1,12,-1,9,-1,14,-1,10]		[0,-1,4,-1,4,-1,4,-1,4,-1,4]
5	[0,-1,7,-1,12,4,9,11,14,16,10]		[0,-1,4,-1,4,5,4,5,4,5,4]
16
